In [1]:
import xarray as xr

In [2]:
xr.open_dataset('/data/bmoorema/MEOPAR/grid/coordinates_seagrid_SalishSea2.nc')

/home/bmoorema/anaconda3/lib/python3.6/site-packages/xarray/coding/times.py:122: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy cftime.datetime objects instead, reason: dates out of range
  result = decode_cf_datetime(example_value, units, calendar)
/home/bmoorema/anaconda3/lib/python3.6/site-packages/xarray/coding/variables.py:69: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy cftime.datetime objects instead, reason: dates out of range
  return self.func(self.array)


<xarray.Dataset>
Dimensions:     (time: 1, x: 398, y: 898)
Coordinates:
  * time        (time) object 0001-01-01 00:00:00
Dimensions without coordinates: x, y
Data variables:
    nav_lon     (y, x) float32 ...
    nav_lat     (y, x) float32 ...
    time_steps  (time) object ...
    glamt       (time, y, x) float64 ...
    glamu       (time, y, x) float64 ...
    glamv       (time, y, x) float64 ...
    glamf       (time, y, x) float64 ...
    gphit       (time, y, x) float64 ...
    gphiu       (time, y, x) float64 ...
    gphiv       (time, y, x) float64 ...
    gphif       (time, y, x) float64 ...
    e1t         (time, y, x) float64 ...
    e1u         (time, y, x) float64 ...
    e1v         (time, y, x) float64 ...
    e1f         (time, y, x) float64 ...
    e2t         (time, y, x) float64 ...
    e2u         (time, y, x) float64 ...
    e2v         (time, y, x) float64 ...
    e2f         (time, y, x) float64 ...
Attributes:
    history:  [2017-02-09] Swapped e1/e2 variables in

In [4]:
def rotation(glam, gphi, attr):
    return 2 * getattr(np, attr)(np.pi * glam / 180) * np.tan(np.pi * (1 / 4 - gphi / 360))

In [ ]:
gcost = np.zeros_like(glamt)
gcosu = np.zeros_like(glamt)
gcosv = np.zeros_like(glamt)
gcosf = np.zeros_like(glamt)
gsint = np.zeros_like(glamt)
gsinu = np.zeros_like(glamt)
gsinv = np.zeros_like(glamt)
gsinf = np.zeros_like(glamt)

jpj, jpi = np.shape(glamt)

rpi = np.pi
rad = np.pi / 180.

for jj in range(1, jpj - 1):
    for ji in range(1, jpi):
        
        # for t, u, v, f
        lims = (slice(1, jpj-1), slice(1, jpi))
        zxnpt, zynpt = [-rotation(glam[lims], gphi[lims], attr) for attr in ['cos', 'sin']]
        znnpt = zxnpt**2 + zynpt**2
        
        # for v around t glamv[jj-1, ji], gphiv[jj-1, ji]
        # for f around u glamf[jj-1, ji], gphif[jj-1, ji]
        # for f around v glamf[jj, ji-1], gphif[jj, ji-1]
        # for u around f glamv[jj+1, ji], gphiv[jj+1, ji] (glam, gphi)
        for glam, gphi in zip(glamv[jj, ji], gphiv[jj, ji]):
        for glan, gphh in zip(glamv[jj-1, ji], gphiv[jj-1, ji]):
        
        for lims in zip():
            zxvvt, zyvvt = [rotation(glam, gphi, attr) for attr in ['cos', 'sin']]
        znvvt = max(np.sqrt(znnpt * (zxvvt**2 + zyvvt**2)), 1e-14)

        
        # ! cosinus and sinus using scalar and vectorial products
        gsint[jj, ji] = (zxnpt * zyvvt - zynpt * zxvvt) / znvvt
        gcost[jj, ji] = (zxnpt * zxvvt + zynpt * zyvvt) / znvvt

        gsinu[jj, ji] = (zxnpu * zyffu - zynpu * zxffu) / znffu
        gcosu[jj, ji] = (zxnpu * zxffu + zynpu * zyffu) / znffu

        gsinf[jj, ji] = (zxnpf * zyuuf - zynpf * zxuuf) / znuuf
        gcosf[jj, ji] = (zxnpf * zxuuf + zynpf * zyuuf) / znuuf

        # ! (caution, rotation of 90 degres)
        gsinv[jj, ji] =  (zxnpv * zxffv + zynpv * zyffv ) / znffv
        gcosv[jj, ji] = -(zxnpv * zyffv - zynpv * zxffv ) / znffv

# 1:  first row is copied from second row
gsint[:,0] = gsint[:,1]
gcost[:,0] = gcost[:,1]
gsinu[:,0] = gsinu[:,1]
gcosu[:,0] = gcosu[:,1]
gsinv[:,0] = gsinv[:,1]
gcosv[:,0] = gcosv[:,1]
gsinf[:,0] = gsinf[:,1]
gcosf[:,0] = gcosf[:,1]

# 2: last row copied from second last row
gsint[:,-1] = gsint[:,-2]
gcost[:,-1] = gcost[:,-2]
gsinu[:,-1] = gsinu[:,-2]
gcosu[:,-1] = gcosu[:,-2]
gsinv[:,-1] = gsinv[:,-2]
gcosv[:,-1] = gcosv[:,-2]
gsinf[:,-1] = gsinf[:,-2]
gcosf[:,-1] = gcosf[:,-2]

# first column copied from second column
gsint[0,:] = gsint[1,:]
gcost[0,:] = gcost[1,:]
gsinu[0,:] = gsinu[1,:]
gcosu[0,:] = gcosu[1,:]
gsinv[0,:] = gsinv[1,:]
gcosv[0,:] = gcosv[1,:]
gsinf[0,:] = gsinf[1,:]
gcosf[0,:] = gcosf[1,:]

# 4: last column copied from second last column
gsint[-1,:] = gsint[-2,:]
gcost[-1,:] = gcost[-2,:]
gsinu[-1,:] = gsinu[-2,:]
gcosu[-1,:] = gcosu[-2,:]
gsinv[-1,:] = gsinv[-2,:]
gcosv[-1,:] = gcosv[-2,:]
gsinf[-1,:] = gsinf[-2,:]
gcosf[-1,:] = gcosf[-2,:]

return gsint,gcost,gsinu,gcosu,gsinv,gcosv,gsinf,gcosf